In [2]:
import numpy as np
import pandas as pd
import cv2

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import sys
from tqdm import tqdm
from time import time
import shutil

In [3]:
import matplotlib.pyplot as plt

def plot_progress(history):
    loss_tr = []
    loss_vl = []
    acc_tr = []
    acc_vl = []
    loss_tr.append(history.history['loss'])
    loss_vl.append(history.history['val_loss'])
    acc_tr.append(history.history['categorical_accuracy'])
    acc_vl.append(history.history['val_categorical_accuracy'])
    
    plt.plot(acc_tr[0])
    plt.plot(acc_vl[0])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(loss_tr[0])
    plt.plot(loss_vl[0])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [4]:
from sklearn.metrics import classification_report,multilabel_confusion_matrix

In [30]:
inception1 = 0
def report(model):
    prediction = model.predict(test_gen)
    if model == inception1:
        print('incept')
        prediction = prediction[0]
    predicted_labels = np.argmax(prediction, axis=1)
    confusion_mat = multilabel_confusion_matrix(test_gen.labels, predicted_labels)
    print(confusion_mat)
    print(classification_report(test_gen.labels, predicted_labels, digits=5))

In [36]:
train_path = 'training_path/Model_Image/train'
val_path = 'validation_path/cwry-2/Model_Image/val'
test_path = 'testing_path/cwry-2/Model_Image/test'

num_train_samples = 12909
num_test_samples = 2815
num_val_samples = 2741
train_batch_size = 64
val_batch_size = 64


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [37]:
train_steps,val_steps

(202.0, 43.0)

In [38]:
IMAGE_SIZE = 256
def preprocess(img):
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    return img

datagen = ImageDataGenerator(preprocessing_function=lambda img: preprocess(img),
                             rescale=1.0/255)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(val_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

test_gen = datagen.flow_from_directory(test_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)

Found 12909 images belonging to 16 classes.
Found 2741 images belonging to 16 classes.
Found 2815 images belonging to 16 classes.


# DL Models

In [9]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, DepthwiseConv2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras import Model

In [10]:
# Metrics
cat_acc = tf.keras.metrics.CategoricalAccuracy()
auc_roc = tf.keras.metrics.AUC(multi_label=True, num_labels=16)
prec = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

In [11]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


## AlexNet

In [ ]:
def AlexNet(input_shape):
    input = Input(shape=input_shape)
    
    x = Conv2D(96, (11, 11), strides=(4, 4), activation='relu')(input)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(384, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.SpatialDropout2D(0.5)(x)
    x = BatchNormalization()(x)
    x = Conv2D(384, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = BatchNormalization()(x)
    
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(16, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='AlexNet')
    
    return model

In [ ]:
alexnet = AlexNet(input_shape=(256,256,3))
alexnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [ ]:
alexnet.count_params()

In [ ]:
start = time()
alexnet_hist = alexnet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(alexnet_hist.history)
hist_json_file = 'AlexNet_50ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
alexnet.save_weights('AlexNet.hdf5')

In [ ]:
alexnet.evaluate(test_gen)

In [ ]:
report(alexnet)

## VGG

In [ ]:
def conv_2(x,filters,kernel_size,activation):
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

def conv_3(x,filters,kernel_size,activation):
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

def conv_4(x,filters,kernel_size,activation):
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

def FC(x,units,activation):
    x = Flatten()(x) 
    x = Dense(units = units, activation =activation)(x)
    x = Dropout(0.5)(x)
    x = Dense(units = units, activation =activation)(x)
    x = Dropout(0.5)(x)
    x = Dense(units = 16, activation ='softmax')(x)
    return x

### VGG16

In [ ]:
def VGG16(input_shape):
    
    input = Input(shape=input_shape)
    
    filters = 64
    kernel_same = 3
    act = 'relu'
    
    x = conv_2(input,filters,kernel_same,act)
    x = conv_2(x,filters*2,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_3(x,filters*4,kernel_same,act)
    x = conv_3(x,filters*8,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_3(x,filters*8,kernel_same,act)
    
    units = 4096
    
    output = FC(x,units,act)
    
    model = Model(inputs=input, outputs=output, name='VGG16')
    
    return model

In [ ]:
vgg16 = VGG16(input_shape=(256,256,3))
vgg16.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [ ]:
vgg16.count_params()

In [ ]:
start = time()
vgg16_hist = vgg16.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(vgg16_hist.history)
hist_json_file = 'VGG16_50ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
vgg16.evaluate(test_gen)

In [ ]:
report(vgg16)

### VGG19

In [ ]:
def VGG19(input_shape):
    
    input = Input(shape=input_shape)
    
    filters = 64
    kernel_same = 3
    act = 'relu'
    
    x = conv_2(input,filters,kernel_same,act)
    x = conv_2(x,filters*2,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_4(x,filters*4,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.5)(x)
    x = conv_4(x,filters*8,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_4(x,filters*8,kernel_same,act)
    
    units = 4096
    
    output = FC(x,units,act)
    
    model = Model(inputs=input, outputs=output)
    
    return model

In [ ]:
vgg19 = VGG19(input_shape=(256,256,3))
vgg19.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [ ]:
vgg19.count_params()

In [ ]:
start = time()
vgg19_hist = vgg19.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(vgg19_hist.history)
hist_json_file = 'VGG19_50ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = vgg19.evaluate(test_gen)

In [ ]:
report(vgg19)

## MobileNet

In [15]:
def block(x, filters, strides, activation):
    
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = Conv2D(filters=filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    return x

def MobileNet(input_shape):
    
    activation = 'relu'
    filters = 32
    reps = 5
    input = Input(shape =input_shape)
    x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = block(x, filters = filters, strides = 1, activation=activation)
    x = block(x, filters = filters*2, strides = 2, activation=activation)
    x = block(x, filters = filters*2, strides = 1, activation=activation)
    x = block(x, filters = filters*4, strides = 2, activation=activation)
    x = Dropout(0.2)(x)
    x = block(x, filters = filters*4, strides = 1, activation=activation)
    x = block(x, filters = filters*8, strides = 2, activation=activation)
    for _ in range (reps):
         x = block(x, filters = filters*8, strides = 1, activation=activation)
    x = block(x, filters = filters*16, strides = 2, activation=activation)
    x = block(x, filters = filters*16, strides = 1, activation=activation)
    x = Dropout(0.2)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    output = Dense(units = 16, activation = 'softmax')(x)
    model = Model(inputs=input, outputs=output)
    
    return model

In [16]:
mobilenet = MobileNet(input_shape=(256,256,3))
mobilenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [ ]:
mobilenet.count_params()

In [ ]:
start = time()
mobilenet_hist = mobilenet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

In [19]:
hist_df = pd.DataFrame(mobilenet_hist.history)
hist_json_file = 'MobileNet_50ep_histori.json'
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = mobilenet.evaluate(test_gen)

In [ ]:
report(mobilenet)

## MobileNet v2

In [22]:
def inverted_residual_block(x, factor, filters_in, filters_out, strides, activation):
    
    x_add = x
    x = Conv2D(filters=filters_in*factor, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = Conv2D(filters=filters_out, kernel_size=1, strides=1,use_bias=False)(x)
    x = BatchNormalization()(x)
    
    if x_add.shape[-1] == x.shape[-1]:
        x = tf.keras.layers.Add()([x_add,x])
    
    return x

In [23]:
def MobileNet_v2(input_shape=(256,256,3)):
    
    input = Input(shape=input_shape)
    
    x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same', use_bias=False)(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu6')(x)
    
    x = inverted_residual_block(x, factor=1, filters_in=x.shape[-1], filters_out=16, strides=1, activation='relu6')
    x = Dropout(0.2)(x)
    for i in range(2):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=24, strides=s, activation='relu6')
    for i in range(3):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=32, strides=s, activation='relu6')
    for i in range(4):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=64, strides=s, activation='relu6')
    for i in range(3):
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=96, strides=1, activation='relu6')
    for i in range(3):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=160, strides=s, activation='relu6')
    x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=320, strides=1, activation='relu6')
    x = Dropout(0.2)(x)
    
    x = Conv2D(filters=1280, kernel_size=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu6')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    x = Flatten()(x)
    output = Dense(units=16, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output)
    
    return model

In [24]:
mobilenet_v2 = MobileNet_v2(input_shape=(256,256,3))
mobilenet_v2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [25]:
mobilenet_v2.count_params()

2293904

In [26]:
start = time()
mobilenet_v2_hist = mobilenet_v2.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-28 20:50:31.877500: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 136s - loss: 1.7519 - categorical_accuracy: 0.4982 - auc: 0.9160 - precision: 0.9357 - recall: 0.3080 - val_loss: 3.8050 - val_categorical_accuracy: 0.0668 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 136s/epoch - 674ms/step
Epoch 2/50
202/202 - 97s - loss: 0.5897 - categorical_accuracy: 0.8036 - auc: 0.9859 - precision: 0.8933 - recall: 0.6932 - val_loss: 4.4494 - val_categorical_accuracy: 0.0657 - val_auc: 0.4982 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 97s/epoch - 478ms/step
Epoch 3/50
202/202 - 97s - loss: 0.2957 - categorical_accuracy: 0.8996 - auc: 0.9950 - precision: 0.9255 - recall: 0.8757 - val_loss: 5.5957 - val_categorical_accuracy: 0.0657 - val_auc: 0.5281 - val_precision: 0.0657 - val_recall: 0.0657 - 97s/epoch - 482ms/step
Epoch 4/50
202/202 - 99s - loss: 0.1699 - categorical_accuracy: 0.9408 - auc: 0.9980 - precision: 0.9522 - recall: 0.9309 - val_loss: 0.6130 - val_categorical_accuracy: 0.7964 - val_auc: 0.9889 - val_p

In [27]:
hist_df = pd.DataFrame(mobilenet_v2_hist.history)
hist_json_file = 'MobileNe_v2_50ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [28]:
pred = mobilenet_v2.evaluate(test_gen)

2815/2815 [==============================] - 34s 12ms/step - loss: 0.0072 - categorical_accuracy: 0.9979 - auc: 1.0000 - precision: 0.9982 - recall: 0.9975


In [29]:
report(mobilenet_v2)

2815/2815 [==============================] - 23s 8ms/step
[[[2628    1]
  [   0  186]]

 [[2628    1]
  [   0  186]]

 [[2628    1]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2628    1]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   1  185]]

 [[2629    0]
  [   2  184]]

 [[2628    1]
  [   0  186]]

 [[2629    0]
  [   1  185]]

 [[2628    1]
  [   0  186]]

 [[2629    0]
  [   1  185]]

 [[2754    0]
  [   0   61]]

 [[2753    0]
  [   1   61]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       186
           1       0.99      1.00      1.00       186
           2       0.99      1.00      1.00       186
           3       1.00      1.00      1.00       186
           4       1.00      1.00      1.00       186
           5       0.99      1.00      1.00       186
           6       1.00      1.00      1.00       186
           7       1.00      

## MobileNet v3

In [11]:
def bottleneck_block(x, kernel_size, stride, filters_out, exp_size, SE, NL):

    expanded_channels = exp_size
    reduction_ratio = 4
    x_add = x
    
    x = Conv2D(expanded_channels, kernel_size=1, padding='valid')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(NL)(x)
    
    x = DepthwiseConv2D(kernel_size=kernel_size, padding='same', strides=stride)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(NL)(x)
    
    if SE == True:
        # Squeeze and Excitation Layer
        squeeze = tf.keras.layers.GlobalAveragePooling2D()(x)
        excitation = Dense(expanded_channels // reduction_ratio, activation='relu')(squeeze)
        excitation = Dense(expanded_channels, activation='hard_sigmoid')(excitation)
        excitation = tf.keras.layers.Reshape((1, 1, expanded_channels))(excitation)
        x = tf.keras.layers.multiply([x, excitation])

    x = Conv2D(filters_out, kernel_size=1, padding='valid')(x)
    x = BatchNormalization()(x)
    
    if x_add.shape == x.shape:
        x = tf.keras.layers.add([x, x_add])
    
    return x

def efficient_final_layers(x,l=960):
    x = Conv2D(l, kernel_size=1, padding='valid', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('swish')(x)
    
    x = tf.keras.layers.AveragePooling2D(8)(x)
    
    x = Conv2D(1280, kernel_size=1, use_bias=False)(x)
    x = tf.keras.layers.Activation('swish')(x)
    x = Dropout(0.2)(x)
    x = Conv2D(16, kernel_size=1, activation='softmax')(x)
    
    x = tf.squeeze(x, 1)
    x = tf.squeeze(x, 1)
    
    return x

In [12]:
def MobileNet_v3(size='Large',input_shape=(256,256,3)):
    
    input = Input(shape=input_shape)
    
    if size == 'Large':
        x = Conv2D(16, kernel_size=3, padding='same', strides=2, use_bias=False)(input)
        x = BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)

                           # x  k  s  out exp sq&ex   act
        x = bottleneck_block(x, 3, 1, 16, 16, False, 'relu')
        x = bottleneck_block(x, 3, 2, 24, 64, False, 'relu')
        x = bottleneck_block(x, 3, 1, 24, 72, False, 'relu')
        x = Dropout(0.2)(x)

        x = bottleneck_block(x, 5, 2, 40, 72, True, 'relu')
        x = bottleneck_block(x, 5, 1, 40, 120, True, 'relu')
        x = bottleneck_block(x, 5, 1, 40, 120, True, 'relu')

        x = bottleneck_block(x, 3, 2, 80, 240, False, 'swish')
        x = bottleneck_block(x, 3, 1, 80, 200, False, 'swish')
        x = bottleneck_block(x, 3, 1, 80, 184, False, 'swish')
        x = bottleneck_block(x, 3, 1, 80, 184, False, 'swish')
        x = bottleneck_block(x, 3, 1, 112, 480, True, 'swish')
        x = bottleneck_block(x, 3, 1, 112, 672, True, 'swish')
        x = Dropout(0.2)(x)

        x = bottleneck_block(x, 5, 2, 160, 672, True, 'swish')
        x = bottleneck_block(x, 5, 1, 160, 960, True, 'swish')
        x = bottleneck_block(x, 5, 1, 160, 960, True, 'swish')
        

        output = efficient_final_layers(x)

        model = Model(inputs=input, outputs=output, name='MobileNetv3_large')
    if size == 'Small':
        x = Conv2D(16, kernel_size=3, padding='same', strides=2, use_bias=False)(input)
        x = BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)

                           # x  k  s  out exp sq&ex   act
        x = bottleneck_block(x, 3, 2, 16, 16, True, 'relu')
        x = bottleneck_block(x, 3, 2, 24, 72, False, 'relu')
        x = bottleneck_block(x, 3, 1, 24, 88, False, 'relu')
        x = Dropout(0.2)(x)
        
        x = bottleneck_block(x, 5, 2, 40, 96, True, 'swish')
        x = bottleneck_block(x, 5, 1, 40, 240, True, 'swish')
        x = bottleneck_block(x, 5, 1, 40, 240, True, 'swish')
        x = bottleneck_block(x, 5, 1, 48, 120, True, 'swish')
        x = bottleneck_block(x, 5, 1, 48, 144, True, 'swish')
        x = bottleneck_block(x, 5, 2, 96, 288, True, 'swish')
        x = Dropout(0.2)(x)
        x = bottleneck_block(x, 5, 1, 96, 576, True, 'swish')
        x = bottleneck_block(x, 5, 1, 96, 576, True, 'swish')

        output = efficient_final_layers(x,l=576)

        model = Model(inputs=input, outputs=output, name='MobileNetv3_Small')
    
    return model

In [13]:
Lmobilenet_v3 = MobileNet_v3(size='Large',input_shape=(256,256,3))
Lmobilenet_v3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [14]:
Lmobilenet_v3.count_params()

4255374

In [35]:
start = time()
Lmobilenet_v3_hist = Lmobilenet_v3.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-28 22:41:17.146668: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inMobileNetv3_large/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 135s - loss: 1.1827 - categorical_accuracy: 0.6648 - auc: 0.9631 - precision: 0.8957 - recall: 0.5371 - val_loss: 2.9111 - val_categorical_accuracy: 0.0668 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 135s/epoch - 667ms/step
Epoch 2/50
202/202 - 87s - loss: 0.2752 - categorical_accuracy: 0.9009 - auc: 0.9952 - precision: 0.9220 - recall: 0.8801 - val_loss: 3.6212 - val_categorical_accuracy: 0.0668 - val_auc: 0.5105 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 87s/epoch - 428ms/step
Epoch 3/50
202/202 - 87s - loss: 0.1420 - categorical_accuracy: 0.9535 - auc: 0.9982 - precision: 0.9604 - recall: 0.9462 - val_loss: 4.8226 - val_categorical_accuracy: 0.1204 - val_auc: 0.6442 - val_precision: 0.1393 - val_recall: 0.1124 - 87s/epoch - 430ms/step
Epoch 4/50
202/202 - 94s - loss: 0.0893 - categorical_accuracy: 0.9714 - auc: 0.9991 - precision: 0.9742 - recall: 0.9679 - val_loss: 0.0962 - val_categorical_accuracy: 0.9683 - val_auc: 0.9992 - val_p

In [36]:
hist_df = pd.DataFrame(Lmobilenet_v3_hist.history)
hist_json_file = 'LMobileNe_v3_50ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [37]:
pred = Lmobilenet_v3.evaluate(test_gen)

2815/2815 [==============================] - 34s 12ms/step - loss: 0.0064 - categorical_accuracy: 0.9979 - auc: 1.0000 - precision: 0.9979 - recall: 0.9979


In [38]:
report(Lmobilenet_v3)

2815/2815 [==============================] - 27s 9ms/step
[[[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2628    1]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2627    2]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2626    3]
  [   0  186]]

 [[2629    0]
  [   6  180]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2754    0]
  [   0   61]]

 [[2753    0]
  [   0   62]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       186
           1       1.00      1.00      1.00       186
           2       0.99      1.00      1.00       186
           3       1.00      1.00      1.00       186
           4       1.00      1.00      1.00       186
           5       0.99      1.00      0.99       186
           6       1.00      1.00      1.00       186
           7       0.98      

In [15]:
Smobilenet_v3 = MobileNet_v3(size='Small',input_shape=(256,256,3))
Smobilenet_v3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [16]:
Smobilenet_v3.count_params()

1697078

In [17]:
start = time()
Smobilenet_v3_hist = Smobilenet_v3.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-29 11:20:12.529983: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inMobileNetv3_Small/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 181s - loss: 1.4272 - categorical_accuracy: 0.5061 - auc: 0.9207 - precision: 0.8856 - recall: 0.3107 - val_loss: 2.9916 - val_categorical_accuracy: 0.0657 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 181s/epoch - 895ms/step
Epoch 2/50
202/202 - 75s - loss: 0.3291 - categorical_accuracy: 0.8871 - auc: 0.9941 - precision: 0.9201 - recall: 0.8524 - val_loss: 4.0762 - val_categorical_accuracy: 0.0668 - val_auc: 0.5051 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 75s/epoch - 371ms/step
Epoch 3/50
202/202 - 74s - loss: 0.1562 - categorical_accuracy: 0.9482 - auc: 0.9981 - precision: 0.9586 - recall: 0.9395 - val_loss: 5.1023 - val_categorical_accuracy: 0.1142 - val_auc: 0.5843 - val_precision: 0.2085 - val_recall: 0.1040 - 74s/epoch - 367ms/step
Epoch 4/50
202/202 - 74s - loss: 0.1035 - categorical_accuracy: 0.9663 - auc: 0.9990 - precision: 0.9703 - recall: 0.9628 - val_loss: 0.1856 - val_categorical_accuracy: 0.9376 - val_auc: 0.9989 - val_p

In [18]:
hist_df = pd.DataFrame(Smobilenet_v3_hist.history)
hist_json_file = 'SMobileNe_v3_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [19]:
pred = Smobilenet_v3.evaluate(test_gen)

2815/2815 [==============================] - 34s 12ms/step - loss: 0.0215 - categorical_accuracy: 0.9929 - auc: 1.0000 - precision: 0.9929 - recall: 0.9929


In [20]:
report(Smobilenet_v3)

2815/2815 [==============================] - 22s 8ms/step
[[[2629    0]
  [   8  178]]

 [[2629    0]
  [   0  186]]

 [[2628    1]
  [   1  185]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2620    9]
  [   1  185]]

 [[2625    4]
  [   1  185]]

 [[2629    0]
  [   3  183]]

 [[2626    3]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2626    3]
  [   4  182]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   2  184]]

 [[2754    0]
  [   0   61]]

 [[2753    0]
  [   0   62]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       186
           1       1.00      1.00      1.00       186
           2       0.99      0.99      0.99       186
           3       1.00      1.00      1.00       186
           4       1.00      1.00      1.00       186
           5       0.95      0.99      0.97       186
           6       0.98      0.99      0.99       186
           7       1.00      

## ResNet under 50

In [15]:
def identity_block(x, filter):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    
    return x

def convolutional_block(x, filter):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same', strides=2)(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    
    x_skip = Conv2D(filters=filter, kernel_size=1, strides=2)(x_skip)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    return x

In [16]:
def ResNet(layers=18, input_shape=(256,256,3)):
    
    filters = 64
    act = 'relu'
    
    input = Input(shape=input_shape)
    #x = tf.keras.layers.ZeroPadding2D((3,3))(x_input)
    
    x = Conv2D(filters=filters, kernel_size=(7,7), strides=2)(input)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(act)(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    if layers == 18:
        block_layers = [2,2,2,2]
    if layers == 34:
        block_layers = [3,4,6,3]
    for i in range(4):
        if i == 0:
            for j in range(block_layers[i]):
                x = identity_block(x, filters)
        else:
            filters = filters*2
            x = convolutional_block(x, filters)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filters)
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    output = Dense(units=16, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='ResNet18')
    
    return model

In [17]:
resnet18 = ResNet(input_shape=(256,256,3))
resnet18.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [18]:
resnet18.count_params()

11195536

In [19]:
start = time()
resnet18_hist = resnet18.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-29 14:54:44.671959: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inResNet18/dropout_3/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 206s - loss: 0.5763 - categorical_accuracy: 0.8168 - auc: 0.9843 - precision: 0.9010 - recall: 0.7639 - val_loss: 3.1902 - val_categorical_accuracy: 0.0657 - val_auc: 0.5657 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 206s/epoch - 1s/step
Epoch 2/50
202/202 - 79s - loss: 0.0978 - categorical_accuracy: 0.9703 - auc: 0.9986 - precision: 0.9735 - recall: 0.9663 - val_loss: 3.2894 - val_categorical_accuracy: 0.0660 - val_auc: 0.6648 - val_precision: 0.1187 - val_recall: 0.0241 - 79s/epoch - 391ms/step
Epoch 3/50
202/202 - 79s - loss: 0.0491 - categorical_accuracy: 0.9857 - auc: 0.9994 - precision: 0.9869 - recall: 0.9842 - val_loss: 1.2111 - val_categorical_accuracy: 0.6359 - val_auc: 0.9670 - val_precision: 0.6771 - val_recall: 0.5991 - 79s/epoch - 389ms/step
Epoch 4/50
202/202 - 79s - loss: 0.0399 - categorical_accuracy: 0.9873 - auc: 0.9996 - precision: 0.9884 - recall: 0.9866 - val_loss: 0.2123 - val_categorical_accuracy: 0.9358 - val_auc: 0.9976 - val_precision: 0

In [20]:
hist_df = pd.DataFrame(resnet18_hist.history)
hist_json_file = 'ResNet18_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [21]:
pred = resnet18.evaluate(test_gen)

2815/2815 [==============================] - 33s 12ms/step - loss: 0.0176 - categorical_accuracy: 0.9950 - auc: 0.9997 - precision: 0.9950 - recall: 0.9950


In [22]:
report(resnet18)

2815/2815 [==============================] - 15s 5ms/step
[[[2628    1]
  [   0  186]]

 [[2629    0]
  [   1  185]]

 [[2629    0]
  [   1  185]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2623    6]
  [   0  186]]

 [[2629    0]
  [   1  185]]

 [[2629    0]
  [   1  185]]

 [[2629    0]
  [   6  180]]

 [[2628    1]
  [   4  182]]

 [[2628    1]
  [   0  186]]

 [[2625    4]
  [   0  186]]

 [[2628    1]
  [   0  186]]

 [[2754    0]
  [   0   61]]

 [[2753    0]
  [   0   62]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       186
           1       1.00      0.99      1.00       186
           2       1.00      0.99      1.00       186
           3       1.00      1.00      1.00       186
           4       1.00      1.00      1.00       186
           5       0.97      1.00      0.98       186
           6       1.00      0.99      1.00       186
           7       1.00      

In [23]:
resnet34 = ResNet(layers=34,input_shape=(256,256,3))
resnet34.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [24]:
resnet34.count_params()

21314832

In [25]:
start = time()
resnet34_hist = resnet34.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-29 16:28:38.258383: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inResNet18/dropout_5/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 117s - loss: 0.7384 - categorical_accuracy: 0.8053 - auc: 0.9815 - precision: 0.8839 - recall: 0.7611 - val_loss: 3.7026 - val_categorical_accuracy: 0.0668 - val_auc: 0.5124 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 117s/epoch - 581ms/step
Epoch 2/50
202/202 - 80s - loss: 0.1338 - categorical_accuracy: 0.9564 - auc: 0.9978 - precision: 0.9618 - recall: 0.9531 - val_loss: 4.2773 - val_categorical_accuracy: 0.0668 - val_auc: 0.5604 - val_precision: 0.0684 - val_recall: 0.0668 - 80s/epoch - 398ms/step
Epoch 3/50
202/202 - 80s - loss: 0.0737 - categorical_accuracy: 0.9764 - auc: 0.9989 - precision: 0.9788 - recall: 0.9743 - val_loss: 3.4140 - val_categorical_accuracy: 0.2149 - val_auc: 0.8326 - val_precision: 0.2164 - val_recall: 0.1795 - 80s/epoch - 397ms/step
Epoch 4/50
202/202 - 81s - loss: 0.0572 - categorical_accuracy: 0.9819 - auc: 0.9994 - precision: 0.9832 - recall: 0.9810 - val_loss: 0.2173 - val_categorical_accuracy: 0.9252 - val_auc: 0.9978 - val_precision

In [26]:
hist_df = pd.DataFrame(resnet34_hist.history)
hist_json_file = 'ResNet34_50ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [27]:
pred = resnet34.evaluate(test_gen)

2815/2815 [==============================] - 32s 11ms/step - loss: 0.0616 - categorical_accuracy: 0.9805 - auc: 0.9996 - precision: 0.9815 - recall: 0.9776


In [28]:
report(resnet34)

2815/2815 [==============================] - 21s 7ms/step
[[[2629    0]
  [   1  185]]

 [[2629    0]
  [   0  186]]

 [[2626    3]
  [   1  185]]

 [[2629    0]
  [   4  182]]

 [[2629    0]
  [   6  180]]

 [[2619   10]
  [   1  185]]

 [[2628    1]
  [  24  162]]

 [[2629    0]
  [  10  176]]

 [[2618   11]
  [   5  181]]

 [[2629    0]
  [   2  184]]

 [[2608   21]
  [   1  185]]

 [[2629    0]
  [   0  186]]

 [[2626    3]
  [   0  186]]

 [[2748    6]
  [   0   61]]

 [[2753    0]
  [   0   62]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       186
           1       1.00      1.00      1.00       186
           2       0.98      0.99      0.99       186
           3       1.00      0.98      0.99       186
           4       1.00      0.97      0.98       186
           5       0.95      0.99      0.97       186
           6       0.99      0.87      0.93       186
           7       1.00      

## ResNet above 50

In [29]:
def BN_identity_block(x, filter):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(1,1), padding='valid')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter*4, kernel_size=(1,1), padding='valid')(x)
    x = BatchNormalization(axis=3)(x)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    
    return x

def BN_convolutional_block(x, filter, s):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(1,1), padding='valid', strides=s)(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter*4, kernel_size=(1,1), padding='valid')(x)
    x = BatchNormalization(axis=3)(x)
    
    x_skip = Conv2D(filters=filter*4, kernel_size=1, strides=s, padding='valid')(x_skip)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    return x

In [30]:
def BN_ResNet(input_shape=(256,256,3)):
    
    filters = 64
    act = 'relu'
    
    x_input = Input(shape=input_shape)
    
    x = Conv2D(filters=filters, kernel_size=(7,7), strides=2)(x_input)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(act)(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    block_layers = [3,4,6,3]
    for i in range(4):
        if i == 0:
            x = BN_convolutional_block(x=x, filter=filters, s=1)
            for j in range(block_layers[i]-1):
                x = BN_identity_block(x, filters)
        else:
            filters = filters*2
            x = BN_convolutional_block(x, filters, s=2)
            for j in range(block_layers[i] - 1):
                x = BN_identity_block(x, filters)
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    output = Dense(units=16, activation='softmax')(x)
    
    model = Model(inputs=x_input, outputs=output, name='ResNet50')
    
    return model

In [31]:
resnet50 = BN_ResNet(input_shape=(256,256,3))
resnet50.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [32]:
resnet50.count_params()

23605136

In [33]:
start = time()
resnet50_hist = resnet50.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-29 17:47:47.369825: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inResNet50/dropout_7/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 153s - loss: 1.1826 - categorical_accuracy: 0.6797 - auc: 0.9591 - precision: 0.8141 - recall: 0.6167 - val_loss: 5.4174 - val_categorical_accuracy: 0.0657 - val_auc: 0.5560 - val_precision: 0.0657 - val_recall: 0.0657 - 153s/epoch - 757ms/step
Epoch 2/50
202/202 - 110s - loss: 0.3020 - categorical_accuracy: 0.8943 - auc: 0.9935 - precision: 0.9090 - recall: 0.8840 - val_loss: 6.6139 - val_categorical_accuracy: 0.0660 - val_auc: 0.5029 - val_precision: 0.0811 - val_recall: 0.0551 - 110s/epoch - 543ms/step
Epoch 3/50
202/202 - 109s - loss: 0.1555 - categorical_accuracy: 0.9480 - auc: 0.9971 - precision: 0.9541 - recall: 0.9442 - val_loss: 7.3525 - val_categorical_accuracy: 0.0890 - val_auc: 0.5694 - val_precision: 0.0982 - val_recall: 0.0719 - 109s/epoch - 541ms/step
Epoch 4/50
202/202 - 110s - loss: 0.1171 - categorical_accuracy: 0.9634 - auc: 0.9977 - precision: 0.9660 - recall: 0.9608 - val_loss: 3.0098 - val_categorical_accuracy: 0.4199 - val_auc: 0.8291 - val_precision: 0

In [34]:
hist_df = pd.DataFrame(resnet50_hist.history)
hist_json_file = 'ResNet50_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [35]:
pred = resnet50.evaluate(test_gen)

2815/2815 [==============================] - 37s 13ms/step - loss: 0.0317 - categorical_accuracy: 0.9897 - auc: 1.0000 - precision: 0.9915 - recall: 0.9890


In [36]:
report(resnet50)

2815/2815 [==============================] - 26s 9ms/step
[[[2628    1]
  [   5  181]]

 [[2629    0]
  [   0  186]]

 [[2628    1]
  [   2  184]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2614   15]
  [   0  186]]

 [[2628    1]
  [   0  186]]

 [[2628    1]
  [  19  167]]

 [[2620    9]
  [   0  186]]

 [[2628    1]
  [   0  186]]

 [[2629    0]
  [   2  184]]

 [[2629    0]
  [   1  185]]

 [[2629    0]
  [   0  186]]

 [[2754    0]
  [   0   61]]

 [[2753    0]
  [   0   62]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       186
           1       1.00      1.00      1.00       186
           2       0.99      0.99      0.99       186
           3       1.00      1.00      1.00       186
           4       1.00      1.00      1.00       186
           5       0.93      1.00      0.96       186
           6       0.99      1.00      1.00       186
           7       0.99      

## DenseNet

In [37]:
def convolutional_layer(x, filters, kernel=1, strides=1):
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = Conv2D(filters, kernel_size=kernel, strides=strides, padding='same')(x)
    return x

def dense_block(x, layers, filters):
    for _ in range(layers):
        x_d = convolutional_layer(x, 4*filters)
        x_d = convolutional_layer(x_d, filters, kernel=3)
        x = tf.keras.layers.Concatenate()([x_d,x])
    return x

def transition_layer(x):
    x = convolutional_layer(x, filters=x.shape[-1]//2)
    x = tf.keras.layers.AveragePooling2D(2, strides=2, padding='same')(x)
    return x

In [39]:
def denseNet121(input_shape):
    input = Input(shape=input_shape)
    
    x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same')(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = MaxPool2D(3, strides=2, padding='same')(x)
    
    for l in [6,12,24,16]:
        d = dense_block(x, layers=l, filters=32)
        x = Dropout(0.2)(x)
        x = transition_layer(d)
    x = Dropout(0.5)(x)
        
    x = tf.keras.layers.GlobalAveragePooling2D()(d)
    x = Flatten()(x)
    output = Dense(units=16, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='DenseNet')
    
    return model

In [40]:
densenet = denseNet121(input_shape=(256,256,3))
densenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [41]:
densenet.count_params()

7060048

In [42]:
start = time()
densenet_hist = densenet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50
202/202 - 211s - loss: 0.5773 - categorical_accuracy: 0.8573 - auc: 0.9921 - precision: 0.9621 - recall: 0.7610 - val_loss: 3.7687 - val_categorical_accuracy: 0.0657 - val_auc: 0.4978 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 211s/epoch - 1s/step
Epoch 2/50
202/202 - 104s - loss: 0.0781 - categorical_accuracy: 0.9778 - auc: 0.9994 - precision: 0.9821 - recall: 0.9736 - val_loss: 4.3804 - val_categorical_accuracy: 0.0657 - val_auc: 0.5243 - val_precision: 0.0657 - val_recall: 0.0657 - 104s/epoch - 515ms/step
Epoch 3/50
202/202 - 104s - loss: 0.0519 - categorical_accuracy: 0.9849 - auc: 0.9994 - precision: 0.9865 - recall: 0.9831 - val_loss: 0.7099 - val_categorical_accuracy: 0.7468 - val_auc: 0.9954 - val_precision: 0.7920 - val_recall: 0.6833 - 104s/epoch - 515ms/step
Epoch 4/50
202/202 - 112s - loss: 0.0257 - categorical_accuracy: 0.9915 - auc: 0.9999 - precision: 0.9925 - recall: 0.9905 - val_loss: 0.0392 - val_categorical_accuracy: 0.9869 - val_auc: 0.9998 - 

In [43]:
hist_df = pd.DataFrame(densenet_hist.history)
hist_json_file = 'densenet_50ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [44]:
pred = densenet.evaluate(test_gen)

2815/2815 [==============================] - 53s 18ms/step - loss: 0.0039 - categorical_accuracy: 0.9989 - auc: 0.9998 - precision: 0.9989 - recall: 0.9989


In [45]:
report(densenet)

2815/2815 [==============================] - 45s 15ms/step
[[[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   3  183]]

 [[2629    0]
  [   0  186]]

 [[2627    2]
  [   0  186]]

 [[2628    1]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2754    0]
  [   0   61]]

 [[2753    0]
  [   0   62]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       186
           1       1.00      1.00      1.00       186
           2       1.00      1.00      1.00       186
           3       1.00      1.00      1.00       186
           4       1.00      1.00      1.00       186
           5       1.00      0.98      0.99       186
           6       1.00      1.00      1.00       186
           7       0.99     

## Inception v1

In [46]:
def inception_module(x,filters_1x1,filters_3x3_reduce,filters_3x3,
    filters_5x5_reduce,filters_5x5,filters_pool_proj,name=None):

    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)
    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)
    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)
    output = tf.keras.layers.Concatenate(axis=3, name=name)([conv_1x1, conv_3x3, conv_5x5, pool_proj])

    return output

In [53]:
def inception_v1(input_shape=(256,256,3)):
    
    input = Input(shape=input_shape)
    
    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(input)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
    x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)
    
    x = inception_module(x,filters_1x1=64,filters_3x3_reduce=96,
     filters_3x3=128,filters_5x5_reduce=16,filters_5x5=32,filters_pool_proj=32,name='inception_3a')
    x = Dropout(0.2)(x)
    
    x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,
     filters_3x3=192,filters_5x5_reduce=32,filters_5x5=96,filters_pool_proj=64,name='inception_3b')
    x = Dropout(0.2)(x)
    
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)
    x = inception_module(x,filters_1x1=192,filters_3x3_reduce=96,
    filters_3x3=208,filters_5x5_reduce=16,filters_5x5=48,filters_pool_proj=64, name='inception_4a')
    x = Dropout(0.2)(x)
    
    x1 = tf.keras.layers.AveragePooling2D((5, 5), strides=3)(x)
    x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
    x1 = Flatten()(x1)
    x1 = Dense(1024, activation='relu')(x1)
    x1 = Dropout(0.7)(x1)
    x1 = Dense(16, activation='softmax', name='auxilliary_output_1')(x1)
    
    x = inception_module(x,filters_1x1=160,filters_3x3_reduce=112,
    filters_3x3=224,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4b')
    x = Dropout(0.2)(x)
    
    x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,
     filters_3x3=256,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4c')
    x = Dropout(0.2)(x)
    
    x = inception_module(x,filters_1x1=112,filters_3x3_reduce=144,
     filters_3x3=288,filters_5x5_reduce=32,filters_5x5=64,filters_pool_proj=64,name='inception_4d')
    x = Dropout(0.2)(x)
    
    x2 = tf.keras.layers.AveragePooling2D((5, 5), strides=3)(x)
    x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1024, activation='relu')(x2)
    x2 = Dropout(0.7)(x2)
    x2 = Dense(16, activation='softmax', name='auxilliary_output_2')(x2)
    
    x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,
     filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_4e')
    
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)
    x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,
     filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_5a')
    
    x = inception_module(x, filters_1x1=384,filters_3x3_reduce=192,
     filters_3x3=384,filters_5x5_reduce=48,filters_5x5=128,filters_pool_proj=128,name='inception_5b')
    
    x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
    x = Dropout(0.4)(x)
    x = Dense(16, activation='softmax', name='output')(x)
    
    model = Model(inputs=input, outputs=[x, x1, x2], name='inception_v1')
    
    return model

In [54]:
inception1 = inception_v1(input_shape=(256,256,3))
inception1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=['categorical_crossentropy','categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1.0,0.3,0.3], metrics=[cat_acc,auc_roc,prec,recall])

In [55]:
start = time()
inception1_hist = inception1.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

Epoch 1/50


2023-08-29 21:28:58.238672: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ininception_v1/dropout_32/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


202/202 - 110s - loss: 3.6889 - output_loss: 2.2788 - auxilliary_output_1_loss: 2.3809 - auxilliary_output_2_loss: 2.3196 - output_categorical_accuracy: 0.1965 - output_auc: 0.7378 - output_precision: 0.8817 - output_recall: 0.0820 - auxilliary_output_1_categorical_accuracy: 0.1879 - auxilliary_output_1_auc: 0.7139 - auxilliary_output_1_precision: 0.9602 - auxilliary_output_1_recall: 0.0635 - auxilliary_output_2_categorical_accuracy: 0.1987 - auxilliary_output_2_auc: 0.7274 - auxilliary_output_2_precision: 0.8916 - auxilliary_output_2_recall: 0.0784 - val_loss: 3.0792 - val_output_loss: 1.9235 - val_auxilliary_output_1_loss: 1.9773 - val_auxilliary_output_2_loss: 1.8751 - val_output_categorical_accuracy: 0.3539 - val_output_auc: 0.8935 - val_output_precision: 0.9444 - val_output_recall: 0.0992 - val_auxilliary_output_1_categorical_accuracy: 0.3057 - val_auxilliary_output_1_auc: 0.8674 - val_auxilliary_output_1_precision: 1.0000 - val_auxilliary_output_1_recall: 0.0992 - val_auxilliary_

In [56]:
hist_df = pd.DataFrame(inception1_hist.history)
hist_json_file = 'inception1_50ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [57]:
pred = inception1.evaluate(test_gen)

2815/2815 [==============================] - 45s 16ms/step - loss: 4.0962 - output_loss: 3.3179 - auxilliary_output_1_loss: 0.2102 - auxilliary_output_2_loss: 2.3842 - output_categorical_accuracy: 0.7854 - output_auc: 0.9136 - output_precision: 0.7865 - output_recall: 0.7851 - auxilliary_output_1_categorical_accuracy: 0.9155 - auxilliary_output_1_auc: 0.9977 - auxilliary_output_1_precision: 0.9245 - auxilliary_output_1_recall: 0.9087 - auxilliary_output_2_categorical_accuracy: 0.8270 - auxilliary_output_2_auc: 0.9330 - auxilliary_output_2_precision: 0.8272 - auxilliary_output_2_recall: 0.8263


In [58]:
report(inception1)

2815/2815 [==============================] - 22s 8ms/step
incept
[[[2543   86]
  [  32  154]]

 [[2621    8]
  [   0  186]]

 [[2629    0]
  [  42  144]]

 [[2593   36]
  [   0  186]]

 [[2629    0]
  [ 100   86]]

 [[2621    8]
  [ 144   42]]

 [[2497  132]
  [   1  185]]

 [[2534   95]
  [  15  171]]

 [[2571   58]
  [  52  134]]

 [[2627    2]
  [   4  182]]

 [[2629    0]
  [  89   97]]

 [[2629    0]
  [   6  180]]

 [[2613   16]
  [  63  123]]

 [[2591  163]
  [   0   61]]

 [[2753    0]
  [  56    6]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0       0.64      0.83      0.72       186
           1       0.96      1.00      0.98       186
           2       1.00      0.77      0.87       186
           3       0.84      1.00      0.91       186
           4       1.00      0.46      0.63       186
           5       0.84      0.23      0.36       186
           6       0.58      0.99      0.74       186
           7       0.6

# Best model

In [40]:
def convolutional_layer(x, filters, kernel=1, strides=1):
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = Conv2D(filters, kernel_size=kernel, strides=strides, padding='same')(x)
    return x

def dense_block(x, layers, filters):
    for _ in range(layers):
        x_d = convolutional_layer(x, 4*filters)
        x_d = convolutional_layer(x_d, filters, kernel=3)
        x = tf.keras.layers.Concatenate()([x_d,x])
    return x

def transition_layer(x):
    x = convolutional_layer(x, filters=x.shape[-1]//2)
    x = tf.keras.layers.AveragePooling2D(2, strides=2, padding='same')(x)
    return x

def denseNet121(input_shape):
    input = Input(shape=input_shape)
    
    x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same')(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = MaxPool2D(3, strides=2, padding='same')(x)
    
    for l in [6,12,24,16]:
        d = dense_block(x, layers=l, filters=32)
        x = Dropout(0.2)(x)
        x = transition_layer(d)
    x = Dropout(0.2)(x)
        
    x = tf.keras.layers.GlobalAveragePooling2D()(d)
    x = Flatten()(x)
    output = Dense(units=16, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='DenseNet')
    
    return model

densenet = denseNet121(input_shape=(256,256,3))
densenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,prec,recall])

In [27]:
def scheduler(epoch, lr):
    if epoch < 40:
        return lr
    elif epoch < 60:
        lr = 1e-4 * tf.math.exp(-0.8)
        return lr
    else:
        lr = 7e-5 * tf.math.exp(-0.9)
        return lr

call = tf.keras.callbacks.LearningRateScheduler(
    scheduler, verbose=0
)

start = time()
densenet_hist = densenet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=70, verbose=2, callbacks=[call])
end = time()
print("===================================")
print(f'70 epochs in {(end-start)/60} min')

Epoch 1/70
202/202 - 196s - loss: 0.5431 - categorical_accuracy: 0.9155 - auc: 0.9970 - precision: 0.9815 - recall: 0.8583 - val_loss: 3.3377 - val_categorical_accuracy: 0.0657 - val_auc: 0.5137 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 1.0000e-04 - 196s/epoch - 968ms/step
Epoch 2/70
202/202 - 104s - loss: 0.0732 - categorical_accuracy: 0.9791 - auc: 0.9995 - precision: 0.9824 - recall: 0.9753 - val_loss: 3.8419 - val_categorical_accuracy: 0.1452 - val_auc: 0.5447 - val_precision: 0.2661 - val_recall: 0.0106 - lr: 1.0000e-04 - 104s/epoch - 516ms/step
Epoch 3/70
202/202 - 106s - loss: 0.0474 - categorical_accuracy: 0.9858 - auc: 0.9997 - precision: 0.9876 - recall: 0.9842 - val_loss: 1.0739 - val_categorical_accuracy: 0.6151 - val_auc: 0.9795 - val_precision: 0.6646 - val_recall: 0.5465 - lr: 1.0000e-04 - 106s/epoch - 526ms/step
Epoch 4/70
202/202 - 104s - loss: 0.0362 - categorical_accuracy: 0.9888 - auc: 0.9996 - precision: 0.9898 - recall: 0.9872 - val_loss: 0.0406 -

In [28]:
densenet.evaluate(test_gen)

2815/2815 [==============================] - 53s 19ms/step - loss: 4.5614e-04 - categorical_accuracy: 0.9996 - auc: 1.0000 - precision: 0.9996 - recall: 0.9996


[0.000456141191534698,
 0.9996447563171387,
 1.0,
 0.9996447563171387,
 0.9996447563171387]

In [31]:
report(densenet)

2815/2815 [==============================] - 41s 15ms/step
[[[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   1  185]]

 [[2628    1]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2754    0]
  [   0   61]]

 [[2753    0]
  [   0   62]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000       186
           1    1.00000   1.00000   1.00000       186
           2    1.00000   1.00000   1.00000       186
           3    1.00000   1.00000   1.00000       186
           4    1.00000   1.00000   1.00000       186
           5    1.00000   1.00000   1.00000       186
           6    1.00000   1.00000   1.00000       186
           7    1.00000   0.

In [32]:
densenet.save_weights("DenseNet121_weights.hdf5")

In [41]:
start = time()
densenet_hist = densenet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=80, verbose=2)
end = time()
print("===================================")
print(f'80 epochs in {(end-start)/60} min')

Epoch 1/80
202/202 - 195s - loss: 0.5807 - categorical_accuracy: 0.8572 - auc: 0.9918 - precision: 0.9694 - recall: 0.7628 - val_loss: 3.6599 - val_categorical_accuracy: 0.0657 - val_auc: 0.5256 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 195s/epoch - 967ms/step
Epoch 2/80
202/202 - 104s - loss: 0.0887 - categorical_accuracy: 0.9736 - auc: 0.9993 - precision: 0.9774 - recall: 0.9699 - val_loss: 4.2092 - val_categorical_accuracy: 0.0657 - val_auc: 0.5320 - val_precision: 0.0820 - val_recall: 0.0657 - 104s/epoch - 515ms/step
Epoch 3/80
202/202 - 104s - loss: 0.0390 - categorical_accuracy: 0.9890 - auc: 0.9997 - precision: 0.9905 - recall: 0.9875 - val_loss: 1.9356 - val_categorical_accuracy: 0.4075 - val_auc: 0.9488 - val_precision: 0.4841 - val_recall: 0.3386 - 104s/epoch - 516ms/step
Epoch 4/80
202/202 - 104s - loss: 0.0353 - categorical_accuracy: 0.9892 - auc: 0.9998 - precision: 0.9901 - recall: 0.9880 - val_loss: 0.0298 - val_categorical_accuracy: 0.9920 - val_auc: 0.9998

In [42]:
densenet.evaluate(test_gen)

2815/2815 [==============================] - 52s 18ms/step - loss: 5.9628e-04 - categorical_accuracy: 0.9996 - auc: 1.0000 - precision: 0.9996 - recall: 0.9996


[0.0005962767172604799,
 0.9996447563171387,
 1.0,
 0.9996447563171387,
 0.9996447563171387]

In [43]:
report(densenet)

2815/2815 [==============================] - 43s 15ms/step
[[[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   1  185]]

 [[2628    1]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2629    0]
  [   0  186]]

 [[2754    0]
  [   0   61]]

 [[2753    0]
  [   0   62]]

 [[2541    0]
  [   0  274]]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000       186
           1    1.00000   1.00000   1.00000       186
           2    1.00000   1.00000   1.00000       186
           3    1.00000   1.00000   1.00000       186
           4    1.00000   1.00000   1.00000       186
           5    1.00000   1.00000   1.00000       186
           6    1.00000   1.00000   1.00000       186
           7    1.00000   0.